In [2]:
import pandas as pd

In [3]:
file_path = 'C:/Users/Shrutika/OneDrive/Desktop/anylist project/Python SAS Internship/Project-2 SAS & PY/Credit card transactions - Project - 2.csv'
df = pd.read_csv(file_path)

## Top 5 cities with highest spends and their percentage contribution of total credit card spends

In [4]:
df_sorted = df.sort_values(by='Amount', ascending=False)
top_5_cities = df_sorted.head(5)
total_spends = df['Amount'].sum()
top_5_cities['Percentage_Contribution'] = (top_5_cities['Amount'] / total_spends) * 100
print("Top 5 Cities with Highest Spends:")
print(top_5_cities[['City', 'Amount', 'Percentage_Contribution']])

C:\Users\Shrutika\AppData\Local\Temp\ipykernel_9044\2155135667.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_5_cities['Percentage_Contribution'] = (top_5_cities['Amount'] / total_spends) * 100


Top 5 Cities with Highest Spends:
                      City  Amount  Percentage_Contribution
80   Greater Mumbai, India  998077                 0.024494
134           Delhi, India  996754                 0.024461
254       Ahmedabad, India  996291                 0.024450
183       Ahmedabad, India  995634                 0.024434
330           Delhi, India  994537                 0.024407


# Highest spend month and amount spent

In [5]:
df['Date'] = pd.to_datetime(df['Date'])
grouped_data = df.groupby(['Card Type', df['Date'].dt.month])['Amount'].sum().reset_index()
# Find the row with the highest spend for each 'CardType'
idx = grouped_data.groupby('Card Type')['Amount'].idxmax()

# Select the rows with the highest spend for each 'CardType'
highest_spend_data = grouped_data.loc[idx]


In [6]:
print("Highest Spend Month and Amount Spent for Each Card Type:")
print(highest_spend_data[['Card Type', 'Date', 'Amount']])

Highest Spend Month and Amount Spent for Each Card Type:
    Card Type  Date     Amount
0        Gold     1  110146204
12   Platinum     1  112784373
33  Signature    10  115694418
38     Silver     3  113547912


## Transaction details reaches a cumulative of 1000000 total spends 

In [7]:
df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')

# Group by 'CardType' and accumulate the 'AmountSpent'
df['CumulativeSpends'] = df.groupby('Card Type')['Amount'].cumsum()

# Filter rows where cumulative spends reach 1000000
filtered_data = df[df['CumulativeSpends'] >= 1000000]

In [8]:
print("Transaction Details for Each Card Type with Cumulative Spends >= 1000000:")
print(filtered_data)

Transaction Details for Each Card Type with Cumulative Spends >= 1000000:
       index                   City       Date  Card Type Exp Type Gender  \
15        15  Greater Mumbai, India 2014-11-28   Platinum    Bills      F   
16        16  Greater Mumbai, India 2014-06-14  Signature    Bills      F   
17        17  Greater Mumbai, India 2015-03-30       Gold    Bills      F   
18        18  Greater Mumbai, India 2014-03-15   Platinum    Bills      F   
19        19  Greater Mumbai, India 2013-11-09   Platinum    Bills      F   
...      ...                    ...        ...        ...      ...    ...   
26047  26047         Kolkata, India 2014-06-22     Silver   Travel      F   
26048  26048            Pune, India 2014-08-03  Signature   Travel      M   
26049  26049       Hyderabad, India 2015-01-16     Silver   Travel      M   
26050  26050          Kanpur, India 2014-09-14     Silver   Travel      M   
26051  26051       Hyderabad, India 2013-10-19  Signature    Bills      M   

 

## Find city which had lowest percentage spend for gold card type

In [9]:

# Filter data for gold card type
gold_data = df[df['Card Type'] == 'Gold']

# Calculate the percentage spend for each city
gold_data['PercentageSpend'] = (gold_data['Amount'] / gold_data.groupby('City')['Amount'].transform('sum')) * 100

# Find the city with the lowest percentage spend for gold card type
lowest_percentage_city = gold_data.groupby('City')['PercentageSpend'].min().idxmin()

C:\Users\Shrutika\AppData\Local\Temp\ipykernel_9044\644706094.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gold_data['PercentageSpend'] = (gold_data['Amount'] / gold_data.groupby('City')['Amount'].transform('sum')) * 100


In [10]:
print("City with Lowest Percentage Spend for Gold Card Type:", lowest_percentage_city)

City with Lowest Percentage Spend for Gold Card Type: Greater Mumbai, India


## Print 3 columns: city, highest_expense_type , lowest_expense_type

In [11]:
grouped_data = df.groupby(['City', 'Exp Type'])['Amount'].sum().reset_index()

# Find the row with the highest spend for each 'City'
idx_max = grouped_data.groupby('City')['Amount'].idxmax()

# Find the row with the lowest spend for each 'City'
idx_min = grouped_data.groupby('City')['Amount'].idxmin()

# Select the rows with the highest and lowest spend for each 'City'
highest_expense_data = grouped_data.loc[idx_max]
lowest_expense_data = grouped_data.loc[idx_min]

# Merge the two DataFrames on 'City'
result_data = pd.merge(highest_expense_data[['City', 'Exp Type']], lowest_expense_data[['City', 'Exp Type']],
                       on='City', suffixes=('_highest', '_lowest'))

In [12]:
print("City, Highest Expense Type, Lowest Expense Type:")
print(result_data)

City, Highest Expense Type, Lowest Expense Type:
                 City Exp Type_highest Exp Type_lowest
0     Achalpur, India          Grocery   Entertainment
1     Adilabad, India            Bills            Food
2    Adityapur, India             Food         Grocery
3        Adoni, India            Bills   Entertainment
4        Adoor, India             Fuel           Bills
..                ...              ...             ...
981    Zaidpur, India             Food           Bills
982    Zamania, India    Entertainment            Fuel
983       Zira, India    Entertainment         Grocery
984   Zirakpur, India             Fuel   Entertainment
985  Zunheboto, India          Grocery   Entertainment

[986 rows x 3 columns]


# Percentage contribution of spends by females for each expense type

In [13]:
female_data = df[df['Gender'] == 'F']

# Group by 'ExpenseType' and calculate the total amount spent by females for each expense type
total_spends_by_expense_type = female_data.groupby('Exp Type')['Amount'].sum()

total_spends_by_all_females = female_data['Amount'].sum()

percentage_contribution = (total_spends_by_expense_type / total_spends_by_all_females) * 100

result_data = pd.DataFrame({
    'Exp Type': percentage_contribution.index,
    'PercentageContribution': percentage_contribution.values
})

In [14]:
print("Percentage Contribution of Spends by Females for Each Expense Type:")
print(result_data)


Percentage Contribution of Spends by Females for Each Expense Type:
        Exp Type  PercentageContribution
0          Bills               26.301753
1  Entertainment               16.263617
2           Food               20.533035
3           Fuel               17.788077
4        Grocery               16.580292
5         Travel                2.533227


## Highest month over month growth in Jan-2014

In [15]:
df['Date'] = pd.to_datetime(df['Date'])

# Filter data for January 2014
january_2014_data = df[df['Date'] == '01-Jan-14']

# Group by 'CardType' and 'ExpenseType', summing the 'AmountSpent'
grouped_data = january_2014_data.groupby(['Card Type', 'Exp Type'])['Amount'].sum().reset_index()

# Sort the DataFrame by 'AmountSpent' in descending order
sorted_data = grouped_data.sort_values(by='Amount', ascending=False)

# Select the row with the highest amount spent
highest_growth_data = sorted_data.head(1)

In [16]:
print("Card and Expense Type Combination with Highest Month-Over-Month Growth in Jan-2014:")
print(highest_growth_data.to_string(index=False))

Card and Expense Type Combination with Highest Month-Over-Month Growth in Jan-2014:
Card Type Exp Type  Amount
Signature    Bills  764353


## Highest total spend to total no of transaction’s ratio

In [17]:
df['Date'] = pd.to_datetime(df['Date'])

# Filter data for weekends (Saturday and Sunday)
weekend_data = df[df['Date'].dt.dayofweek.isin([5, 6])]

# Group by 'City', summing the 'AmountSpent' and counting the number of transactions
grouped_data = weekend_data.groupby('City').agg({'Amount': 'sum', 'index': 'count'}).reset_index()

# Calculate the ratio of total spend to total number of transactions
grouped_data['Ratio'] = grouped_data['Amount'] / grouped_data['index']

# Find the row with the highest ratio
highest_ratio_data = grouped_data.loc[grouped_data['Ratio'].idxmax()]

In [18]:
print("City with Highest Spend to Transaction Ratio During Weekends:")
print(highest_ratio_data[['City', 'Ratio']])

City with Highest Spend to Transaction Ratio During Weekends:
City     Sonepur, India
Ratio          299905.0
Name: 657, dtype: object


# 500th transaction after first transaction in that city

In [19]:
df['Date'] = pd.to_datetime(df['Date'])

# Group by 'City' and calculate the time difference between transactions
df['DaysSinceFirstTransaction'] = df.groupby('City')['Date'].transform(lambda x: (x - x.min()).dt.days)

# Filter data for cities with at least 500 transactions
cities_500_transactions = df[df.groupby('City')['index'].transform('count') >= 500]

# Find the city that took the least number of days to reach its 500th transaction after the first transaction
result_data = cities_500_transactions.groupby('City')['DaysSinceFirstTransaction'].min().idxmin()


In [20]:
print("City that took the least days to reach its 500th transaction after the first transaction:", result_data)

City that took the least days to reach its 500th transaction after the first transaction: Ahmedabad, India
